# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770805509700                   -0.52    9.0         
  2   -2.772145785646       -2.87       -1.32    1.0    163ms
  3   -2.772170260912       -4.61       -2.46    1.0    166ms
  4   -2.772170638205       -6.42       -3.15    1.0    214ms
  5   -2.772170722585       -7.07       -4.08    2.0    193ms
  6   -2.772170722745       -9.80       -4.21    1.0    172ms
  7   -2.772170722984       -9.62       -4.70    1.0    203ms
  8   -2.772170723009      -10.60       -5.09    1.0    178ms
  9   -2.772170723015      -11.24       -6.22    1.0    200ms
 10   -2.772170723015      -12.82       -6.19    2.0    219ms
 11   -2.772170723015      -13.24       -6.84    1.0    183ms
 12   -2.772170723015      -14.01       -7.62    2.0    235ms
 13   -2.772170723015   +  -14.21       -7.75    1.0    187ms
 14   -2.772170723015   +  -15.05       -8.16    1.0    214ms
n     Ene

1.773557979689211

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770789297183                   -0.52    9.0         
  2   -2.772057410798       -2.90       -1.32    1.0    166ms
  3   -2.772083133636       -4.59       -2.52    1.0    171ms
  4   -2.772083368767       -6.63       -3.32    1.0    210ms
  5   -2.772083414757       -7.34       -3.78    2.0    195ms
  6   -2.772083417668       -8.54       -4.67    1.0    174ms
  7   -2.772083417798       -9.89       -4.98    2.0    259ms
  8   -2.772083417810      -10.93       -5.68    1.0    182ms
  9   -2.772083417811      -12.10       -6.09    2.0    216ms
 10   -2.772083417811      -13.05       -7.21    1.0    206ms
 11   -2.772083417811   +  -14.21       -7.78    1.0    184ms
 12   -2.772083417811      -14.03       -8.87    2.0    218ms

Polarizability via ForwardDiff:       1.7725349677726303
Polarizability via finite difference: 1.773557979689211
